In [1]:
import cv2 # OpenCV for image overlay

###################################
##### PROVIDE INPUTS HERE !!! #####
###################################

# Background frames prefix (the larger images)
pre1 = "./3/frames_vtk/darcyl16."
suf1 = ".png"
# Front frames prefix (the smaller images)
pre2 = "./3/frames_node/nodes."
suf2 = ".png"
# Output
preout = "./3/frames_overlay/overlay_"
sufout = ".png"

In [2]:
####################
# This fuction lays img2 (smaller) on top of img1 (larger)
# (xoffset, yoffset) indicates position of img2's Upper-Left corner on img1 
####################
def create_overlay_image(bgfile, fgfile, outfile, isFixHeight, scale, xoffset, yoffset, isTest=False):
    
    # Load image files 
    img1 = cv2.imread(bgfile) # The larger image (background)
    img2 = cv2.imread(fgfile) # The smaller image (foreground)
    
    # Get image dimensions
    (h1, w1, d1) = img1.shape
    (h2, w2, d2) = img2.shape
    if isTest:
        print("Img1 (Height, Width, Depth): ", h1, w1, d1)
        print("Img2 (Height, Width, Depth): ", h2, w2, d2)
    
    # Scale img2 (fix eigher heitht or width)
    if isFixHeight:
        h2new = int(h1 * scale)
        w2new = int(w2 * (h1 * scale / h2))
    else:
        w2new = int(w1 * scale)
        h2new = int(h2 * (w1 * scale / w2))
    img2new = cv2.resize(img2, (w2new, h2new))
    
    if isTest:
        print("New img2 dimensions: ", img2new.shape[0], img2new.shape[1], img2new.shape[2])
        
    # Lay img2new over img1
    x1 = int(w1 * xoffset)
    y1 = int(h1 * yoffset)
    y2 = y1 + img2new.shape[0]
    x2 = x1 + img2new.shape[1]
    img3 = img1
    img3[y1:y2, x1:x2] = img2new
    
    # write output
    cv2.imwrite(outfile,img3)

    
####################
# This fuction sets (changes) pixel color of the input image
####################
def change_pixel_color(imgfile, outfile, xoffset, yoffset, xrange, yrange, colorR, colorB, colorG):
    img = cv2.imread(imgfile)
    (h, w, d) = img.shape
    
    y1 = int(h * yoffset)
    x1 = int(w * xoffset)
    y2 = y1 + int(h * yrange)
    x2 = x1 + int(w * xrange)
    
    if (y2-y1 > 0) and (x2-x1 > 0):
        img[y1:y2, x1:x2] = (colorR, colorB, colorG)
    elif (y2-y1 <= 0) and (x2-x1 > 0):
        img[:, x1:x2] = (colorR, colorB, colorG)
    elif (y2-y1 > 0) and (x2-x1 <= 0):
        img[y1:y2, :] = (colorR, colorB, colorG)
    else:
        img[:,:] = (colorR, colorB, colorG)
        
    cv2.imwrite(outfile, img)
        

In [3]:
####################
# Test for 1 image
####################

bgfile = pre1 + '0000' + suf1
fgfile = pre2 + '0000' + suf2
outfile = preout + '0000' + sufout

# Cover the black belt
xoffset = 0.7
yoffset = 0
xrange = 0.01
yrange = 0
colorR, colorB, colorG = 255, 255, 255

change_pixel_color(bgfile, outfile, xoffset, yoffset, xrange, yrange, colorR, colorB, colorG)

# Overlay image
isFixHeight = True
scale = 0.3
xoffset1 = 0.41
yoffset1 = 0.65

create_overlay_image(outfile, fgfile, outfile, isFixHeight, scale, xoffset1, yoffset1, True)


Img1 (Height, Width, Depth):  992 1852 3
Img2 (Height, Width, Depth):  1080 1920 3
New img2 dimensions:  297 529 3


In [4]:
for i in range(1,401):
    idx = "%04d" % i
    bgfile = pre1 + idx + suf1
    fgfile = pre2 + idx + suf2
    outfile = preout + idx + sufout
    #1. cover the black belt
    change_pixel_color(bgfile, outfile, xoffset, yoffset, xrange, yrange, colorR, colorB, colorG)
    #2. Overlay image
    create_overlay_image(outfile, fgfile, outfile, isFixHeight, scale, xoffset1, yoffset1, False)

In [5]:
# Make video from png frames (ffmpeg)
#------------------------------------
# ffmpeg [input options] -i input_file [output options] output_file
#
# !!NOTE: Order matters, do not change order arbitrarily
#
# -f: input format
# -r: fps (frame rate per second)
# -s: resolution in pixel
# -i: input files
# -vcodec: video format
# -crf: quality, number between 15-25 is really good
# -pix_fmt: pixel format

# Input part
cmd ='ffmpeg -f image2 -i ' + preout + '%04d' + sufout
# Output part (must come after input part)
cmd = cmd + ' -r 4 -s 1920x1080 -vcodec libx264 -crf 20 -pix_fmt yuv420p ./3/demo.mp4'
print(cmd)

ffmpeg -f image2 -i ./3/frames_overlay/overlay_%04d.png -r 4 -s 1920x1080 -vcodec libx264 -crf 20 -pix_fmt yuv420p ./3/demo.mp4
